In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [5]:
datosML=pd.read_csv(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\MLproyecto\Data\Processed\datosML.csv",index_col=False)
datosML

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,CANAL_NUM,tipodeobra
0,2,0,4.0,1,1,1
1,3,0,2.0,2,2,2
2,3,0,4.0,2,2,1
3,4,0,2.0,2,2,4
4,4,0,1.0,3,1,1
...,...,...,...,...,...,...
1332,4,0,5.0,1,3,1
1333,4,0,5.0,9,1,1
1334,4,0,5.0,1,3,1
1335,4,0,5.0,3,3,2


In [6]:
datosML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MES            1337 non-null   int64  
 1   Resultado      1337 non-null   int64  
 2   COMERCIAL_NUM  1337 non-null   float64
 3   TELEOP_NUM     1337 non-null   int64  
 4   CANAL_NUM      1337 non-null   int64  
 5   tipodeobra     1337 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 62.8 KB


In [7]:
datosML.describe()

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,CANAL_NUM,tipodeobra
count,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000
mean,5.899776,0.225879,3.382947,3.305161,2.027674,3.359761
std,3.392993,0.418316,1.866810,2.782470,0.952454,1.606022
min,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,0.000000,2.000000,1.000000,1.000000,2.000000
50%,6.000000,0.000000,3.000000,2.000000,2.000000,4.000000
75%,9.000000,0.000000,5.000000,6.000000,3.000000,5.000000
max,12.000000,1.000000,7.000000,11.000000,4.000000,5.000000


In [8]:
datosML.corr()

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,CANAL_NUM,tipodeobra
MES,1.000000,-0.016207,0.043288,-0.071522,0.087715,0.086977
Resultado,-0.016207,1.000000,0.091392,-0.018108,0.048173,-0.028575
COMERCIAL_NUM,0.043288,0.091392,1.000000,-0.029575,0.011716,-0.027511
TELEOP_NUM,-0.071522,-0.018108,-0.029575,1.000000,-0.706171,-0.118888
CANAL_NUM,0.087715,0.048173,0.011716,-0.706171,1.000000,0.080097
tipodeobra,0.086977,-0.028575,-0.027511,-0.118888,0.080097,1.000000


# Comprobación de equilibrio de target

In [9]:
datosML["Resultado"].value_counts()

0    1035
1     302
Name: Resultado, dtype: int64

In [10]:
#Dado que se se observa un desequilibrio en nuestro target, realizaremos un oversampling para solucionarlo

from imblearn.over_sampling import RandomOverSampler

oversampler = RandomOverSampler()
X_oversampled, y_oversampled = oversampler.fit_resample(datosML.drop("Resultado", axis=1), datosML["Resultado"])

# Creamos un DataFrame con el oversampling
datosoversampled = pd.DataFrame(X_oversampled, columns=datosML.drop("Resultado", axis=1).columns)
datosoversampled["Resultado"] = y_oversampled


In [11]:
y_oversampled.value_counts()

0    1035
1    1035
Name: Resultado, dtype: int64

# Pre modelos machine learning

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

In [13]:
X= X_oversampled
y= y_oversampled

In [14]:

validation_size = 0.20
seed = 7
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,
                                                                    y,
                                                                    test_size=validation_size,
                                                                    random_state=seed)


## Fast check modelos

In [15]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

clf= LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions= clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [00:02<00:00, 11.01it/s]


In [16]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
RandomForestClassifier             0.81               0.82     0.82      0.81   
ExtraTreesClassifier               0.81               0.81     0.81      0.81   
ExtraTreeClassifier                0.79               0.80     0.80      0.79   
DecisionTreeClassifier             0.78               0.79     0.79      0.78   
BaggingClassifier                  0.78               0.79     0.79      0.78   
LabelPropagation                   0.78               0.79     0.79      0.78   
LabelSpreading                     0.77               0.78     0.78      0.77   
XGBClassifier                      0.77               0.78     0.78      0.77   
LGBMClassifier                     0.74               0.74     0.74      0.74   
NuSVC                              0.72               0.73     0.73      0.72   
KNeighborsClassifier        

## MODELO 1: LOGISTIC REGRESION

In [17]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [18]:
prediction= model.predict(X_test)


In [19]:
accuracyregression=  model.score(X_test, y_test)
accuracyregression

0.5483091787439613

In [20]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)

[0.57831325 0.56626506 0.54819277 0.47590361 0.52409639 0.57228916
 0.55757576 0.48484848 0.57575758 0.44242424]


## MODELO 2: RANDOM TREE

In [21]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

dtr = DecisionTreeRegressor()
dtr.fit(X_train,y_train)

DecisionTreeRegressor()

In [22]:
dtrprediction= dtr.predict(X_test)

In [23]:
dtraccuracy= dtr.score (X_test, y_test)
dtraccuracy

0.39317058272644234

In [24]:
dtraccscores = cross_val_score(dtr, X, y, cv=5)
dtraccscores

array([-1.40278451, -1.24622704, -0.90231264, -0.24661851,  0.        ])

## MODELO 3: BAGGING

In [25]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


dtc = DecisionTreeClassifier()
modelbag = BaggingClassifier(base_estimator=dtc, n_estimators=100, random_state=seed)

In [26]:
modelbag.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100,
                  random_state=7)

In [27]:
bagpred = modelbag.predict(X_test)
bagcuracy=modelbag.score(X_test, y_test)
bagcuracy

0.8140096618357487

In [28]:
bagscores = cross_val_score(modelbag, X, y, cv=5)
bagscores

array([0.67391304, 0.65700483, 0.72705314, 0.65458937, 0.70772947])

In [29]:
mean_score = bagscores.mean()
print("Mean Accuracy:", mean_score)

Mean Accuracy: 0.6840579710144927


In [30]:
# pruebas extra para validar la fiabilidad

from sklearn.metrics import precision_score, recall_score, f1_score

In [31]:
#precision 
precision = precision_score(y_test, bagpred)
precision

0.7552742616033755

In [32]:
#recall

recall = recall_score(y_test, bagpred)
recall

0.9040404040404041

In [33]:
#f1 score
f1 = f1_score(y_test, bagpred)
f1

0.8229885057471266

## MODELO 4: Ada Boost

In [34]:
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier()
adaboost.fit(X_train, y_train)


AdaBoostClassifier()

In [35]:
adapred=adaboost.predict(X_test)


In [36]:
adaaccuracy=adaboost.score(X_test, y_test)
adaaccuracy

0.5917874396135265

In [37]:
adaboostscores = cross_val_score(adaboost, X, y, cv=5)
adaboostscores

array([0.34541063, 0.38164251, 0.42512077, 0.42995169, 0.50483092])

## MODELO 5: RANDOM FOREST

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


In [39]:
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [40]:
y_pred = rf_classifier.predict(X_test)


In [41]:
accuracy_randomforest = accuracy_score(y_test, y_pred)
accuracy_randomforest

0.8091787439613527

In [42]:
#Cross validation
forestscores = cross_val_score(rf_classifier, X, y, cv=5)
forestscores

array([0.6763285 , 0.70048309, 0.75845411, 0.69806763, 0.7294686 ])

In [43]:
mean_forest=forestscores.mean()
mean_forest

0.7125603864734299

## Modelo 6: Extra Trees Classifier 

In [44]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier()

# Entrenar el modelo con tus datos de entrenamiento
clf.fit(X_train, y_train)

# Realizar predicciones en tus datos de prueba
y_pred = clf.predict(X_test)



In [64]:
# Calcular la precisión del modelo
accuracy = clf.score(X_test, y_test)
accuracy

0.8067632850241546

In [46]:
#cross validation

extratreecross=cross_val_score(clf, X, y, cv=5)
extratreecross.mean()

0.7265700483091786

## Ensemble

<p> Combinamos los tres modelos que nos ofrecen mejores resultados de cara a testear si su ensemble nos ofrece un mejor score, establecemos votación mayoritaria como criterio</p>

In [62]:
from sklearn.ensemble import VotingClassifier

# Definir los modelos base
model1 = ExtraTreesClassifier()
model2 = RandomForestClassifier()
model3 = BaggingClassifier()

# Crear el ensemble con votación mayoritaria
ensemble = VotingClassifier(estimators=[('model1', model1), ('model2', model2), ('model3', model3)], voting='hard')

# Entrenar el ensemble
ensemble.fit(X_train, y_train)

# Obtener las predicciones del ensemble
predictions = ensemble.predict(X_test)


In [66]:
scoreensemble = ensemble.score(X_test, y_test)
scoreensemble


0.8188405797101449

In [67]:
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, f1_score

# Obtener las predicciones del ensemble model
predictions = ensemble.predict(X_test)

# Calcular el Balanced Accuracy
balanced_acc = balanced_accuracy_score(y_test, predictions)

# Calcular el ROC AUC
roc_auc = roc_auc_score(y_test, predictions)

# Calcular el F1 Score
f1 = f1_score(y_test, predictions)

print("Balanced Accuracy:", balanced_acc)
print("ROC AUC:", roc_auc)
print("F1 Score:", f1)


Balanced Accuracy: 0.8217592592592593
ROC AUC: 0.8217592592592593
F1 Score: 0.8243559718969554


## MEJOR MODELO: Ensemble (ExtraTreesClassifier, RandomForestClassifier y BaggingClassifier - Voting)

In [61]:
import joblib

joblib.dump(ensemble, 'new_model.pkl')

['new_model.pkl']

# FEATURE IMPORTANCE

<p>Extraigo el Feature importance de los dos modelos con mejor resultado para obtener una perspectiva de la importancia de cada variable en el resultado del modelo</p>

In [56]:
#feature importance de Extratrees


# Obtener la importancia de las características
importances = clf.feature_importances_

# Crear un DataFrame con las características y su importancia
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Ordenar las características por su importancia descendente
feature_importances = feature_importances.sort_values('Importance', ascending=False)

# Imprimir el ranking de importancia de las características
print(feature_importances)


         Feature  Importance
0            MES        0.40
1  COMERCIAL_NUM        0.21
2     TELEOP_NUM        0.19
4     tipodeobra        0.15
3      CANAL_NUM        0.05


In [57]:
# Obtener la importancia de las características
importances = rf_classifier.feature_importances_

# Crear un DataFrame con las características y su importancia
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Ordenar las características por su importancia descendente
feature_importances = feature_importances.sort_values('Importance', ascending=False)

# Imprimir el ranking de importancia de las características
print(feature_importances)


         Feature  Importance
0            MES        0.37
1  COMERCIAL_NUM        0.21
2     TELEOP_NUM        0.20
4     tipodeobra        0.16
3      CANAL_NUM        0.05
